# Import libraries

In [ ]:
#potential resources
# https://blog.data.nasdaq.com/api-for-interest-rate-data
# https://api-ninjas.com/api/interestrate
# https://data.oecd.org/interest/long-term-interest-rates.htm

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import requests

# Define the API endpoint
api_url = "https://api.api-ninjas.com/v1/interestrate"

# Define the headers for the API request
key = 'QjuPz1UUafFgZrrj+hOaog==jks4hAXt1kivxWFS'

headers = {
    'X-Api-Key': key  # replace 'YOUR_API_KEY' with your actual API key
}

# Define the parameters for the API request
params = {
    'country': 'Denmark'  # replace 'Mexico' with 'Switzerland' to get Swiss rates
}

# Make the API request
response = requests.get(api_url, headers=headers, params=params)

# Check the response
if response.status_code == 200:
    print(response.json())
else:
    print("Error:", response.status_code, response.text)


{'central_bank_rates': [{'central_bank': 'Danish Central Bank', 'country': 'Denmark', 'rate_pct': 3.75, 'last_updated': '09-15-2023'}], 'non_central_bank_rates': [{'name': 'Euribor - 1 week', 'rate_pct': 3.899, 'last_updated': '02-23-2024'}, {'name': 'USD LIBOR - overnight', 'rate_pct': 5.06157, 'last_updated': '06-30-2023'}, {'name': 'ESTER', 'rate_pct': 3.909, 'last_updated': '02-22-2024'}, {'name': 'SARON', 'rate_pct': 1.692129, 'last_updated': '02-22-2024'}, {'name': 'SONIA', 'rate_pct': 5.188, 'last_updated': '02-21-2024'}, {'name': 'TONAR', 'rate_pct': -0.006, 'last_updated': '02-21-2024'}]}


In [7]:
import requests
key = 'QjuPz1UUafFgZrrj+hOaog==jks4hAXt1kivxWFS'
api_url = 'https://api.api-ninjas.com/v1/interestrate'
response = requests.get(api_url, headers={'X-Api-Key': key})
if response.status_code == requests.codes.ok:
    print(response.text)
else:
    print("Error:", response.status_code, response.text)

{"central_bank_rates": [{"central_bank": "Australian Central Bank", "country": "Australia", "rate_pct": 4.35, "last_updated": "11-08-2023"}, {"central_bank": "British Central Bank", "country": "United Kingdom", "rate_pct": 5.25, "last_updated": "08-03-2023"}, {"central_bank": "Chilean Central Bank", "country": "Chile", "rate_pct": 7.25, "last_updated": "02-01-2024"}, {"central_bank": "Chinese Central Bank", "country": "China", "rate_pct": 3.45, "last_updated": "08-21-2023"}, {"central_bank": "Czech Central Bank", "country": "Czech Republic", "rate_pct": 6.25, "last_updated": "02-08-2024"}, {"central_bank": "Danish Central Bank", "country": "Denmark", "rate_pct": 3.75, "last_updated": "09-15-2023"}, {"central_bank": "Mexican Central Bank", "country": "Mexico", "rate_pct": 11.25, "last_updated": "03-30-2023"}, {"central_bank": "New Zealand Central Bank", "country": "New Zealand", "rate_pct": 5.5, "last_updated": "05-24-2023"}, {"central_bank": "Norwegian Central Bank", "country": "Norway

In [12]:
import requests
from datetime import datetime, timedelta

key = 'QjuPz1UUafFgZrrj+hOaog==jks4hAXt1kivxWFS'
api_url = 'https://api.api-ninjas.com/v1/interestrate'

response = requests.get(api_url, headers={'X-Api-Key': key})
if response.status_code == requests.codes.ok:
    data = response.json()
    # Get the current date
    current_date = datetime.now()
    # Calculate the date two years ago from today
    two_years_ago = current_date - timedelta(days=2*365)
    
    # Filter the data for the last two years
    filtered_central_bank_rates = [item for item in data['central_bank_rates'] if datetime.strptime(item['last_updated'], '%m-%d-%Y') >= two_years_ago]
    filtered_non_central_bank_rates = [item for item in data['non_central_bank_rates'] if datetime.strptime(item['last_updated'], '%m-%d-%Y') >= two_years_ago]
    
    print("Filtered Central Bank Rates:", filtered_central_bank_rates)
    print("Filtered Non-Central Bank Rates:", filtered_non_central_bank_rates)
else:
    print("Error:", response.status_code, response.text)



Filtered Central Bank Rates: [{'central_bank': 'Australian Central Bank', 'country': 'Australia', 'rate_pct': 4.35, 'last_updated': '11-08-2023'}, {'central_bank': 'British Central Bank', 'country': 'United Kingdom', 'rate_pct': 5.25, 'last_updated': '08-03-2023'}, {'central_bank': 'Chilean Central Bank', 'country': 'Chile', 'rate_pct': 7.25, 'last_updated': '02-01-2024'}, {'central_bank': 'Chinese Central Bank', 'country': 'China', 'rate_pct': 3.45, 'last_updated': '08-21-2023'}, {'central_bank': 'Czech Central Bank', 'country': 'Czech Republic', 'rate_pct': 6.25, 'last_updated': '02-08-2024'}, {'central_bank': 'Danish Central Bank', 'country': 'Denmark', 'rate_pct': 3.75, 'last_updated': '09-15-2023'}, {'central_bank': 'Mexican Central Bank', 'country': 'Mexico', 'rate_pct': 11.25, 'last_updated': '03-30-2023'}, {'central_bank': 'New Zealand Central Bank', 'country': 'New Zealand', 'rate_pct': 5.5, 'last_updated': '05-24-2023'}, {'central_bank': 'Norwegian Central Bank', 'country': '

# Load data

In [2]:
import os
import PyPDF2
from datetime import datetime

# Specify the folder
folder = './speaches'

# Get a list of all files in the folder
files = os.listdir(folder)

# Filter the list for PDF files
pdf_files = [file for file in files if file.endswith('.pdf')]

# Iterate over the PDF files
for pdf_file in pdf_files:
    # Extract the date from the filename
    country, date_str = pdf_file.split('_')[0], pdf_file.split('_')[1]  # This assumes the date is always at the same position in the filename

    
    try:
        date_str = date_str.split('.')[0] # This assumes the date is always
        date = datetime.strptime(date_str, '%y%m%d')  # Convert the date string to a datetime object
    except ValueError:
        print(f"Error: The date string {date_str} from {pdf_file} does not match the format '%y%m%d'")
    # Open the PDF file
    with open(os.path.join(folder, pdf_file), 'rb') as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Read the content of the PDF
        content = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            content += page.extract_text()

        # Now you can use the content of the PDF and the date as you wish


Error: The date string r231023h from India_r231023h.pdf does not match the format '%y%m%d'
Error: The date string r221118 from Japan_r221118.pdf does not match the format '%y%m%d'
Error: The date string 231115i from Switzerland_231115i.pdf does not match the format '%y%m%d'
Error: The date string and from Topology_and_rates.pdf does not match the format '%y%m%d'
Error: The date string 231115m from USA_231115m.pdf does not match the format '%y%m%d'


In [3]:
import pandas as pd
import os
from datetime import datetime
import PyPDF2

# Initialize an empty dictionary to hold the dataframes
dfs = {}

for pdf_file in os.listdir(folder):
    # Extract the country and date from the filename
    country, date_str = pdf_file.split('_')[0], pdf_file.split('_')[1]

    # Remove the file extension from the date string
    date_str = date_str.split('.')[0]

    try:
        # Convert the date string to a datetime object
        date = datetime.strptime(date_str, '%y%m%d')
    except ValueError:
        print(f"Error: The date string {date_str} from {pdf_file} does not match the format '%y%m%d'")
        continue

    # Open the PDF file
    with open(os.path.join(folder, pdf_file), 'rb') as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Read the content of the PDF
        content = ''
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            content += page.extract_text()

        # If the country is not in the dictionary, add it
        if country not in dfs:
            dfs[country] = pd.DataFrame(columns=['content'])

        # Add the content of the PDF to the dataframe
        dfs[country].loc[date] = content
        
for country, df in dfs.items():
    df.to_parquet(f'{country.lower()}.parquet')

Error: The date string 231023h from India_231023h.pdf does not match the format '%y%m%d'
Error: The date string and from Topology_and_rates.pdf does not match the format '%y%m%d'


In [3]:
# bbc news data
import io
df = pd.read_csv(io.BytesIO(uploaded['bbc-text.csv']))

df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [4]:
df.shape

(2225, 2)

In [5]:
df['category'].value_counts(normalize=True)

sport            0.229663
business         0.229213
politics         0.187416
tech             0.180225
entertainment    0.173483
Name: category, dtype: float64

# Remove Stopwords and create two lists containing articles and labels

In [6]:
articles = []
labels = []

for index, row in df.iterrows():
  labels.append(row['category'])

  token_list = [x for x in row['text'].split() if x not in stopwords.words('english')]

  articles.append(" ".join(token_list))

In [7]:
df.iloc[0, 1]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [8]:
articles[0]

'tv future hands viewers home theatre systems plasma high-definition tvs digital video recorders moving living room way people watch tv radically different five years time. according expert panel gathered annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes. us leading trend programmes content delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices. one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes like us tivo uk sky+ system allow people record store play pause forward wind tv programmes want. essentially technology allows much personalised tv. also built-in high-definition tv sets big business japan us slower take europe lack high-definition programming. people forward wind adverts also forget abiding network channel schedules putting together a-la-carte entertainment. us networks cable satellite companies worried means terms a

# Train test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(articles, labels, random_state = 42, test_size = 0.2,
                                                    stratify = labels)

In [10]:
print(len(X_train), len(X_test))

1780 445


# Tokenization and word indexing

In [12]:
# Define some variables
vocab_size = 5000 # max number of words
embedding_dim = 64 # This is the dimension of the Word Vector
max_length = 200 # maximum length of the sequence
trunc_type = 'post' # where to chop off
padding_type = 'post' # where to put the padding
oov_token = '<OOV>'


In [14]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token = oov_token)

In [16]:
tokenizer.fit_on_texts(X_train)

In [17]:
tokenizer.word_index

{'<OOV>': 1,
 'said': 2,
 'mr': 3,
 'would': 4,
 'year': 5,
 'also': 6,
 'people': 7,
 'new': 8,
 'us': 9,
 'one': 10,
 'could': 11,
 'last': 12,
 'time': 13,
 'first': 14,
 'two': 15,
 'world': 16,
 'government': 17,
 'uk': 18,
 'years': 19,
 'make': 20,
 'best': 21,
 'told': 22,
 'like': 23,
 'film': 24,
 'get': 25,
 'music': 26,
 'made': 27,
 '000': 28,
 'game': 29,
 'back': 30,
 'many': 31,
 'labour': 32,
 'three': 33,
 'well': 34,
 'bbc': 35,
 'set': 36,
 'number': 37,
 '1': 38,
 'added': 39,
 'way': 40,
 'next': 41,
 'take': 42,
 'company': 43,
 '2': 44,
 'says': 45,
 'may': 46,
 'market': 47,
 'election': 48,
 '2004': 49,
 'good': 50,
 'home': 51,
 'england': 52,
 'going': 53,
 'still': 54,
 'show': 55,
 'much': 56,
 'work': 57,
 'games': 58,
 'party': 59,
 'blair': 60,
 'firm': 61,
 'go': 62,
 'top': 63,
 'think': 64,
 'since': 65,
 'second': 66,
 'win': 67,
 '6': 68,
 'say': 69,
 'million': 70,
 'use': 71,
 'public': 72,
 'part': 73,
 'week': 74,
 'want': 75,
 'minister': 76,


# Convert words to number

In [18]:
train_seq = tokenizer.texts_to_sequences(X_train)

# Let's see if text_to_sequence makes sense

In [19]:
X_train[10]

'lewis-francis turns christie mark lewis-francis stepped preparations new season taking advice british sprint icon linford christie. 22-year-old set compete sheffield weekend take maurice greene kim collins birmingham 18 february. training wales getting advice linford christie broadening mind said lewis-francis. sprinter also shed weight since winning relay gold athens games. last year 91kg 86.9kg - hopefully times come said. brought eating right foods cutting snacks. discipline focused doing. still keeping weights work see improvement running. despite playing part britain successful 4x100m relay team lewis-francis still feels frustration missing individual 100m final 2004 olympics. heartbreaking made semi-final personal level achievement. patient build next olympics. goal whatever geared making final.'

In [21]:
print(train_seq[10])

[1821, 3363, 3238, 1, 422, 1821, 3363, 2482, 1, 8, 294, 263, 1411, 87, 4276, 4483, 1, 1, 1066, 5, 105, 36, 1229, 1895, 630, 42, 1, 4725, 2760, 1718, 1523, 416, 419, 739, 162, 408, 1411, 1, 1, 1, 1186, 2, 1821, 3363, 3239, 6, 1, 3629, 65, 347, 3030, 1319, 1003, 58, 12, 5, 1, 1, 1, 1935, 230, 103, 2, 724, 4097, 135, 1, 1392, 1, 2928, 2055, 1, 54, 2105, 1, 57, 80, 2699, 444, 201, 240, 73, 104, 1025, 1, 3030, 146, 1821, 3363, 54, 2483, 1, 1320, 958, 2242, 180, 49, 1566, 1, 27, 2153, 180, 364, 457, 2547, 1, 850, 41, 1566, 740, 2617, 1, 207, 180]


In [22]:
len(train_seq[10])

114

In [23]:
len(train_seq[25])

317

# Let's do padding

When we train neural networks for NLP, we need sequences to be in the same size, that’s why we use padding.

In [24]:
train_padded = pad_sequences(train_seq, maxlen = max_length, padding = padding_type, truncating = trunc_type)

In [25]:
len(train_padded[10])

200

In [26]:
train_padded[10]

array([1821, 3363, 3238,    1,  422, 1821, 3363, 2482,    1,    8,  294,
        263, 1411,   87, 4276, 4483,    1,    1, 1066,    5,  105,   36,
       1229, 1895,  630,   42,    1, 4725, 2760, 1718, 1523,  416,  419,
        739,  162,  408, 1411,    1,    1,    1, 1186,    2, 1821, 3363,
       3239,    6,    1, 3629,   65,  347, 3030, 1319, 1003,   58,   12,
          5,    1,    1,    1, 1935,  230,  103,    2,  724, 4097,  135,
          1, 1392,    1, 2928, 2055,    1,   54, 2105,    1,   57,   80,
       2699,  444,  201,  240,   73,  104, 1025,    1, 3030,  146, 1821,
       3363,   54, 2483,    1, 1320,  958, 2242,  180,   49, 1566,    1,
         27, 2153,  180,  364,  457, 2547,    1,  850,   41, 1566,  740,
       2617,    1,  207,  180,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [28]:
len(train_padded[25])

200

In [29]:
test_seq = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_seq, maxlen = max_length, padding = padding_type, truncating = trunc_type)

# Encode our labels

In [30]:
le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [31]:
y_train[:5]

array([0, 3, 1, 2, 2])

In [32]:
le.classes_

array(['business', 'entertainment', 'politics', 'sport', 'tech'],
      dtype='<U13')

In [33]:
le.inverse_transform([0, 4, 2, 2, 1, 4, 3])

array(['business', 'tech', 'politics', 'politics', 'entertainment',
       'tech', 'sport'], dtype='<U13')

In [34]:
y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

In [35]:
y_train

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

# Define our model

In [42]:
from tensorflow.keras.layers import Bidirectional

In [48]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_length))
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(5, activation = 'softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 200, 64)           320000    
                                                                 
 bidirectional_4 (Bidirecti  (None, 200, 128)          66048     
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 427589 (1.63 MB)
Trainable params: 427589 (1.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train our model

In [50]:
model.fit(train_padded, y_train, validation_data = (test_padded, y_test),
          batch_size = 256, epochs = 10)

Epoch 1/10
7/7 [==============================] - 9s 394ms/step - loss: 1.5923 - accuracy: 0.2798 - val_loss: 1.5700 - val_accuracy: 0.3011
Epoch 2/10
7/7 [==============================] - 1s 182ms/step - loss: 1.5368 - accuracy: 0.3713 - val_loss: 1.5041 - val_accuracy: 0.4404
Epoch 3/10
7/7 [==============================] - 1s 214ms/step - loss: 1.3924 - accuracy: 0.4579 - val_loss: 1.1974 - val_accuracy: 0.5213
Epoch 4/10
7/7 [==============================] - 1s 120ms/step - loss: 0.9883 - accuracy: 0.6747 - val_loss: 0.8326 - val_accuracy: 0.6202
Epoch 5/10
7/7 [==============================] - 1s 155ms/step - loss: 0.7068 - accuracy: 0.7090 - val_loss: 0.6753 - val_accuracy: 0.7371
Epoch 6/10
7/7 [==============================] - 1s 102ms/step - loss: 0.5278 - accuracy: 0.8522 - val_loss: 0.5538 - val_accuracy: 0.8562
Epoch 7/10
7/7 [==============================] - 1s 124ms/step - loss: 0.3529 - accuracy: 0.9534 - val_loss: 0.3824 - val_accuracy: 0.9258
Epoch 8/10
7/7 [====

# Evaluate our model

In [53]:
pred = np.argmax(model.predict(test_padded), axis = 1)

14/14 [==============================] - 0s 27ms/step


In [54]:
pred

array([0, 0, 1, 0, 2, 3, 2, 2, 3, 2, 2, 1, 3, 2, 1, 0, 2, 4, 3, 1, 0, 4,
       3, 3, 4, 3, 1, 2, 4, 3, 2, 2, 0, 3, 1, 3, 1, 4, 1, 3, 4, 2, 1, 3,
       3, 1, 1, 0, 1, 2, 0, 2, 3, 0, 1, 3, 3, 3, 0, 4, 4, 0, 4, 0, 2, 4,
       0, 4, 1, 3, 0, 2, 4, 4, 1, 0, 0, 2, 0, 3, 0, 4, 4, 0, 2, 2, 4, 3,
       4, 2, 0, 4, 1, 3, 3, 3, 2, 2, 2, 0, 1, 0, 2, 4, 3, 1, 0, 3, 3, 3,
       0, 2, 0, 1, 2, 0, 4, 0, 1, 1, 1, 4, 4, 3, 3, 2, 2, 3, 4, 2, 3, 3,
       4, 4, 1, 1, 1, 3, 4, 0, 0, 2, 0, 3, 3, 1, 2, 4, 3, 1, 2, 2, 1, 3,
       3, 3, 4, 3, 2, 3, 3, 3, 3, 3, 4, 0, 1, 0, 1, 0, 2, 1, 4, 3, 3, 2,
       1, 4, 4, 0, 4, 3, 3, 1, 4, 3, 1, 0, 2, 3, 4, 2, 3, 0, 3, 1, 0, 4,
       1, 2, 2, 1, 3, 3, 3, 2, 0, 2, 0, 0, 3, 1, 4, 1, 2, 1, 3, 3, 0, 4,
       0, 3, 4, 4, 3, 4, 0, 0, 0, 0, 1, 2, 4, 3, 4, 2, 4, 3, 2, 3, 3, 2,
       3, 0, 2, 3, 0, 0, 0, 3, 1, 1, 4, 4, 1, 0, 0, 0, 1, 0, 4, 0, 1, 1,
       2, 4, 0, 1, 4, 0, 3, 0, 3, 0, 1, 0, 3, 0, 2, 4, 2, 0, 4, 2, 4, 3,
       2, 4, 3, 3, 3, 0, 0, 1, 2, 0, 3, 0, 2, 2, 4,

In [55]:
le.inverse_transform(pred)

array(['business', 'business', 'entertainment', 'business', 'politics',
       'sport', 'politics', 'politics', 'sport', 'politics', 'politics',
       'entertainment', 'sport', 'politics', 'entertainment', 'business',
       'politics', 'tech', 'sport', 'entertainment', 'business', 'tech',
       'sport', 'sport', 'tech', 'sport', 'entertainment', 'politics',
       'tech', 'sport', 'politics', 'politics', 'business', 'sport',
       'entertainment', 'sport', 'entertainment', 'tech', 'entertainment',
       'sport', 'tech', 'politics', 'entertainment', 'sport', 'sport',
       'entertainment', 'entertainment', 'business', 'entertainment',
       'politics', 'business', 'politics', 'sport', 'business',
       'entertainment', 'sport', 'sport', 'sport', 'business', 'tech',
       'tech', 'business', 'tech', 'business', 'politics', 'tech',
       'business', 'tech', 'entertainment', 'sport', 'business',
       'politics', 'tech', 'tech', 'entertainment', 'business',
       'business', 'p